In [14]:
import numpy as np
from scipy.stats import invgamma
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import time
import plotly.io as pio
from scipy import stats
pio.renderers.default = 'iframe'

In [151]:
class GaussianArm :
    """
    Normal inverse Gamma Prior (=Conjugate Prior for a Normal likelihood) 
    """
    def __init__(self, initial_params, true_params, num_samples_to_compute_posterior=1):
        self.µ, self.α, self.β = initial_params
        self.true_µ, self.true_σ = true_params
        self.var = self.β / (self.α + 1)
        self.num_samples = num_samples_to_compute_posterior
        self.N = 0
        self.observations = []

    def sample_prior(self):
        mean = self.µ
        std = np.sqrt(invgamma(self.α, self.β).rvs(1)[0] / (self.N + self.num_samples))
        return np.random.normal(mean, std, self.num_samples)

    def get_observation(self):
        return np.random.normal(self.true_µ, self.true_σ, self.num_samples)[0]

    def compute_posterior(self, observation):

        self.α = self.α + self.num_samples / 2   
        self.β = self.β + (self.num_samples * self.N / (self.num_samples + self.N)) * (((observation - self.µ)**2)/2)  

        self.N += self.num_samples
        self.observations.append(observation)
        self.µ = np.mean(self.observations)
        self.var = self.β / (self.α + 1)

    def get_params(self):
        return self.µ, self.var ** 0.5

In [25]:
class GaussianTSArm:
    def __init__(self, initial_params, true_params, num_observations=1):
        self.µ, self.v, self.α, self.β = initial_params
        self.num_observations = num_observations
        self.true_µ, self.true_σ = true_params
        self.var = self.β / (self.α + 1)
        self.observations_list = []

    def sample_prior(self):
        mean = self.µ
        normalization_factor = self.v
        if self.v == 0:
            normalization_factor = self.num_observations
        std = np.sqrt(invgamma(self.α, self.β).rvs(1)[0] / normalization_factor)
        return np.random.normal(mean, std, 1)

    def get_observation(self):
        return np.random.normal(self.true_µ, self.true_σ)

    def compute_posterior(self, observations):
        self.µ = (self.v * self.µ + np.sum(observations)) / (self.v + self.num_observations)
        self.v += self.num_observations
        self.α += self.num_observations / 2
        self.β += np.std(observations) * 0.5 * self.num_observations + (
                (self.num_observations * self.v) / (self.num_observations + self.v)) * (
                          ((np.mean(observations) - self.µ) ** 2) / 2)

        self.var = self.β / (self.α + 1)

    def get_params(self):
        return self.µ, self.var ** 0.5

In [30]:
class MultiArmedBandit():
    def __init__(self, arms, theoretical_params=None):
        self.arms = arms
        self.n_iter = 0

    def sample_priors(self):
        return [arm.sample_prior() for arm in self.arms]
        
    def compute_posteriors(self, arm_index, observation):
        self.arms[arm_index].compute_posterior(observation)
        self.n_iter += 1

    def get_params(self):
        return [arm.get_params() for arm in self.arms]
    
    def show(self):
        fig = make_subplots(rows=1, cols=1, subplot_titles=("Beta distributions"))
    
        for i, (mean, std) in enumerate(self.get_params()):
            x = np.linspace(mean - 4*std, mean + 4*std, 200)
            gaussian_pdf = stats.norm.pdf(x, mean, std)
            
            fig.add_trace(go.Scatter(x=x,
                                     y=gaussian_pdf,
                                     mode='lines',
                                     name=f"{i} - Mean={round(mean, 2)}, Std={round(std, 2)} - Num tries {self.arms[i].v}"),
                         row=1, col=1)

        #fig.add_trace(go.Scatter(x=list(range(len(self.regrets))), y=np.cumsum(self.regrets), mode='lines', name='Cumulative regrets'), row=1, col=2)
    
        fig.update_layout(title_text=f"Itération : {self.n_iter}")
        fig.show()
        

In [31]:
means = [0.2, 0.9, 0.5]
stds = [0.02, 0.02, 0.01]

In [32]:
arms = [GaussianTSArm(initial_params=(0.5, 0, 1, 0.3),
                    true_params=(means[i], stds[i]))
        for i in range(len(means))
       ]
mab = MultiArmedBandit(arms)

In [33]:
num_iters = 2000
eval_iters = 50

for i in range(num_iters):
    sampled_values = mab.sample_priors()
    print(sampled_values)
    best_arm_index = np.argmax(sampled_values)
    observation = mab.arms[best_arm_index].get_observation()
    mab.compute_posteriors(best_arm_index, observation)

    if i % eval_iters == eval_iters - 1:
        time.sleep(1)
        mab.show()
        

[array([1.44259342]), array([-1.68972935]), array([0.03107242])]
[array([1.57654293]), array([1.12578919]), array([-0.17822377])]
[array([0.52978861]), array([1.24609793]), array([-1.0327488])]
[array([0.32468136]), array([0.53355757]), array([-0.45687356])]
[array([0.31678635]), array([-0.71072562]), array([1.38944437])]
[array([0.25785304]), array([0.23297522]), array([0.15242897])]
[array([0.77718074]), array([1.26491145]), array([0.66134181])]
[array([-0.7046458]), array([0.76180444]), array([0.00740504])]
[array([0.56307666]), array([0.5478153]), array([-0.35288809])]
[array([0.16837286]), array([1.09688037]), array([0.63403481])]
[array([0.0971935]), array([1.02369184]), array([0.19370843])]
[array([-0.11165905]), array([1.00743415]), array([2.16190842])]
[array([-0.48114281]), array([0.73689054]), array([0.73111708])]
[array([0.14040759]), array([0.54430541]), array([-1.15506935])]
[array([0.73948911]), array([0.68201569]), array([0.38655509])]
[array([0.33035763]), array([0.541

[array([0.28656579]), array([0.96512094]), array([-0.08802297])]
[array([0.07174118]), array([0.84372621]), array([0.02763962])]
[array([0.18687057]), array([1.02292087]), array([0.22888999])]
[array([-0.46844873]), array([0.95106915]), array([0.48489204])]
[array([0.33495281]), array([0.77515553]), array([0.25088834])]
[array([-0.20219907]), array([0.96292771]), array([0.09115167])]
[array([0.80508807]), array([0.93358473]), array([1.18601362])]
[array([0.91720056]), array([0.85912108]), array([0.36703818])]
[array([0.59260135]), array([1.06384449]), array([0.58354333])]
[array([0.00655669]), array([0.9534849]), array([0.09661351])]
[array([0.17308294]), array([0.90338453]), array([0.51481424])]
[array([-0.05658095]), array([0.99807023]), array([0.7378684])]
[array([0.70461642]), array([0.82162778]), array([0.19113419])]
[array([0.62078579]), array([0.92662819]), array([0.11320726])]
[array([0.64031039]), array([0.88561213]), array([0.18989379])]
[array([-0.1682309]), array([0.8485386

[array([0.05762554]), array([0.8400428]), array([0.66358554])]
[array([0.22104869]), array([0.92307788]), array([0.85679774])]
[array([0.24803962]), array([0.94833125]), array([0.40147932])]
[array([-0.37954793]), array([1.07837689]), array([0.69686251])]
[array([0.46641019]), array([0.88183166]), array([0.20038402])]
[array([0.22574487]), array([0.84667739]), array([0.44814664])]
[array([0.01739242]), array([0.86353385]), array([0.47568016])]
[array([0.27533642]), array([0.97592376]), array([0.69652966])]
[array([0.00975047]), array([0.96044135]), array([0.52088682])]
[array([-0.43663977]), array([0.91688132]), array([0.73365218])]
[array([0.30298555]), array([0.95177712]), array([0.3916144])]
[array([-0.16493179]), array([0.99793996]), array([0.37377579])]
[array([0.32263269]), array([0.84151281]), array([0.58414831])]
[array([0.31887973]), array([0.85881433]), array([0.29735556])]
[array([-0.089076]), array([0.91904337]), array([0.4121027])]
[array([-0.21798487]), array([0.94380527]

[array([-0.01886206]), array([0.87833856]), array([0.83877298])]
[array([-0.23721306]), array([0.89907141]), array([0.59237306])]
[array([0.08159444]), array([0.99043662]), array([0.09932866])]
[array([0.12322776]), array([0.87582331]), array([0.58742913])]
[array([0.39507369]), array([0.98831526]), array([0.58674771])]
[array([0.22044485]), array([0.94467444]), array([0.33145997])]
[array([-0.06068507]), array([0.87583566]), array([0.30448543])]
[array([-0.09564975]), array([0.93940251]), array([0.42550321])]
[array([0.05213388]), array([0.7861797]), array([0.26524622])]
[array([0.06800392]), array([0.9656988]), array([0.49962833])]
[array([-0.32799209]), array([0.91184599]), array([0.84174745])]
[array([0.14774409]), array([0.93163346]), array([0.6487552])]
[array([0.35980782]), array([0.93124698]), array([0.43392176])]
[array([0.24721777]), array([0.93795391]), array([0.506589])]
[array([0.19155108]), array([0.90457957]), array([0.86129218])]
[array([-0.17939582]), array([0.93019683

[array([0.12159326]), array([0.88915754]), array([0.81026742])]
[array([0.03539039]), array([0.8713564]), array([0.24283129])]
[array([0.06870017]), array([0.84334706]), array([0.57384417])]
[array([0.28154429]), array([0.86293412]), array([0.33652508])]
[array([0.35120785]), array([0.97537454]), array([0.66230422])]
[array([0.04488917]), array([0.80274743]), array([1.05548766])]
[array([-0.12594891]), array([0.87600014]), array([0.51999724])]
[array([0.31060966]), array([0.86088377]), array([0.70538806])]
[array([0.33822066]), array([0.91678934]), array([0.46056152])]
[array([0.2475393]), array([0.90991046]), array([0.4681032])]
[array([0.31932544]), array([0.93789586]), array([0.68189759])]
[array([-0.00526529]), array([0.93132953]), array([0.67070318])]
[array([-0.24072137]), array([0.89066953]), array([0.36056579])]
[array([-0.06167689]), array([0.92024292]), array([0.61664961])]
[array([-0.34113754]), array([0.89906616]), array([0.63217282])]
[array([0.98211484]), array([0.9178521

...

[array([0.48598244]), array([0.92661857]), array([0.3475801])]
[array([0.40206736]), array([0.89321499]), array([0.37494235])]
[array([0.51648731]), array([0.90531767]), array([0.38334028])]
[array([0.25706004]), array([0.88486016]), array([0.22642323])]
[array([0.44583339]), array([0.92736397]), array([0.6348806])]
[array([0.07348114]), array([0.90435301]), array([0.50276574])]
[array([0.52257434]), array([0.91148733]), array([0.45389909])]
[array([0.19925823]), array([0.90630492]), array([0.38738891])]
[array([0.14790506]), array([0.87324432]), array([0.61542204])]
[array([-0.03046092]), array([0.8962927]), array([0.59856753])]
[array([0.49671067]), array([0.87085277]), array([0.60700514])]
[array([0.22312349]), array([0.8719709]), array([0.36109018])]
[array([0.18754987]), array([0.90181534]), array([0.2757011])]
[array([0.30747205]), array([0.91354407]), array([0.12635571])]
[array([0.21102937]), array([0.92403995]), array([0.4419243])]
[array([0.28114549]), array([0.90912348]), ar

[array([0.58316672]), array([0.87234681]), array([0.58552712])]
[array([0.4158136]), array([0.89671553]), array([0.50997786])]
[array([0.52023654]), array([0.90794954]), array([0.32448495])]
[array([0.04900646]), array([0.93199139]), array([0.6744376])]
[array([0.07253536]), array([0.90224776]), array([0.63039069])]
[array([0.32538632]), array([0.86620528]), array([0.344618])]
[array([0.31226554]), array([0.87017315]), array([0.60314188])]
[array([-0.12292522]), array([0.88709985]), array([0.57094164])]
[array([0.432276]), array([0.90501385]), array([0.41737666])]
[array([0.29544462]), array([0.92248277]), array([0.5331672])]
[array([0.12132644]), array([0.90208738]), array([0.56731855])]
[array([-0.05557157]), array([0.89098362]), array([0.44470605])]
[array([0.40524087]), array([0.88298536]), array([0.45714758])]
[array([0.34032268]), array([0.91533572]), array([0.59862887])]
[array([-0.02207397]), array([0.9274779]), array([0.52857387])]
[array([0.24326956]), array([0.89469897]), ar

[array([0.65851057]), array([0.89903682]), array([0.37070727])]
[array([0.22803416]), array([0.89050434]), array([0.25958477])]
[array([0.0879304]), array([0.92619403]), array([0.6338197])]
[array([0.29238467]), array([0.86499053]), array([0.68315083])]
[array([0.29716511]), array([0.88176708]), array([0.52055275])]
[array([0.48632608]), array([0.90551072]), array([0.54044581])]
[array([0.17627347]), array([0.90666981]), array([0.6194569])]
[array([0.21515562]), array([0.89933627]), array([0.4282426])]
[array([0.23501902]), array([0.8956684]), array([0.27258379])]
[array([0.48320956]), array([0.89927105]), array([0.47601443])]
[array([0.57995063]), array([0.89138703]), array([0.76626299])]
[array([0.01923569]), array([0.90420085]), array([0.49773318])]
[array([0.15237959]), array([0.91973709]), array([0.39537003])]
[array([0.34105679]), array([0.89193081]), array([0.40989988])]
[array([0.27501169]), array([0.90141034]), array([0.63300063])]
[array([0.32849244]), array([0.90982234]), ar

[array([-0.17219974]), array([0.92738825]), array([0.51993954])]
[array([0.5238094]), array([0.90912724]), array([0.59326594])]
[array([0.03314264]), array([0.88454293]), array([0.62500279])]
[array([0.17585551]), array([0.89704588]), array([0.62090399])]
[array([0.42485775]), array([0.87994204]), array([0.43326371])]
[array([0.55687124]), array([0.9170674]), array([0.63026642])]
[array([-0.0053804]), array([0.92700867]), array([0.44217681])]
[array([0.10204106]), array([0.90401508]), array([0.54094712])]
[array([0.10610066]), array([0.91181336]), array([0.4655145])]
[array([0.2636691]), array([0.90008272]), array([0.59469731])]
[array([0.03181886]), array([0.90758796]), array([0.43795375])]
[array([0.35038922]), array([0.91173665]), array([0.46020285])]
[array([0.32741488]), array([0.88429518]), array([0.50221469])]
[array([0.23028568]), array([0.92273986]), array([0.4011795])]
[array([0.23069145]), array([0.89725996]), array([0.5292276])]
[array([0.25832466]), array([0.91095573]), ar

[array([0.21261016]), array([0.88419786]), array([0.60535748])]
[array([0.3879296]), array([0.91583097]), array([0.56005908])]
[array([-0.09132315]), array([0.90357832]), array([0.73861486])]
[array([-0.0755726]), array([0.86516892]), array([0.51416776])]
[array([0.38836369]), array([0.89773827]), array([0.59352994])]
[array([0.150095]), array([0.88549354]), array([0.38700809])]
[array([0.32165774]), array([0.90098107]), array([0.48058007])]
[array([0.0367012]), array([0.89436204]), array([0.35949221])]
[array([0.59000704]), array([0.87277936]), array([0.62382228])]
[array([0.45855144]), array([0.8821457]), array([0.54396261])]
[array([0.23903598]), array([0.89921833]), array([0.55802429])]
[array([0.04915255]), array([0.87681762]), array([0.54443248])]
[array([-0.22164975]), array([0.89938267]), array([0.52024865])]
[array([-0.11310037]), array([0.87542527]), array([0.65287739])]
[array([0.09955236]), array([0.87832129]), array([0.53676975])]
[array([0.20072513]), array([0.88701654]),